In [1]:
import json
import pandas as pd
import numpy as np

import re
import dateutil
from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
 def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name) 
            

In [3]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    return movie

In [4]:
def parse_dollars(s):  
    s = str(s)
    match = re.search(r"\$([\d,.]+)[^\d,.]?.*([mb])il",s) #regex expression 
    if not match: 
        return np.nan 
    digit = float(match[1].replace(",",""))
    try: 
        multiplier=match[2]  
    except IndexError: 
        multiplier = 1.0 
    else: 
        multiplier = {"m":1e6, "b":1e9}[multiplier] 
    return digit*multiplier 
def parse_date(s): 
    if not isinstance(s,list): 
        s=[s] 
    for i in s: 
        try: 
            date = pd.to_datetime(i) 
        except: 
            continue 
        else: 
            return date 
        return np.nan 
def parse_time(s): 
    s = str(s) 
    m = re.search(r"(\d+)", s) 
    if not m: 
        return np.nan 
    else: 
        return pd.to_timedelta(float(m[1]),"min") 

In [5]:
print(parse_date("July 11, 1990"))

1990-07-11 00:00:00


In [6]:
print(parse_time("[original time] 113m"))

0 days 01:53:00


In [7]:
def wkm():
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file) 
    ratings = pd.read_csv(ratings_file) 
    
    # 3. Open the read the Wikipedia data JSON file.
    with open (wiki_file,'r') as f: 
        wiki_movies = json.load(f) 
    wiki_movies = [entry for entry in wiki_movies if "No. of episodes" not in entry] 
    wiki_movies = [clean_movie(entry) for entry in wiki_movies]

    wiki_movies_with_id = []
    imdb_ids = set()
    for entry in wiki_movies:  
        try: 
            m = re.search(r"title/tt(\d{7,8})/", entry["imdb_link"]) 
            imdb_id = m[1]
        except Exception as e:  
            print(f"{entry.get('title', 'N/A')}: {e}")
            continue
        if imdb_id in imdb_ids: 
            continue 
        entry["imdb_id"] = imdb_id
        imdb_ids.add(imdb_id)
        wiki_movies_with_id.append(entry)
        
    wiki_movies_df = pd.DataFrame(wiki_movies_with_id) #Should it just be wiki_movies in the parenthaesis?  
    wiki_movies_df = wiki_movies_df.dropna(how='all',axis=1)
    #box_office = wiki_movies_df["Box office"].dropna()
    wiki_movies_df["Box office"] = wiki_movies_df["Box office"].apply(parse_dollars) 
    wiki_movies_df["Release date"] = wiki_movies_df["Release date"].apply(parse_date) 
    wiki_movies_df["Running time"] = wiki_movies_df["Running time"].apply(parse_time)
    # 5. Return the three DataFrames
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings

In [8]:
file_dir = "//Users/johncurran/Desktop/Rutgers Data Sci Bootcamp/Challenges/ETL Challenge/archive"
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

# 7. Set the three variables in Step 6 equal to the function created in Step 1.
wiki_file, kaggle_file, ratings_file = wkm() #=extract_transform_load() 
wiki_movies_df = wiki_file

/var/folders/9s/jwyj784906ndy4td5h3gqtnw0000gn/T/ipykernel_3037/183294519.py:10: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  wiki_file, kaggle_file, ratings_file = wkm() #=extract_transform_load()


N/A: 'imdb_link'
N/A: 'imdb_link'
A Man Called Sarge: 'imdb_link'
N/A: 'imdb_link'
Side Out: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
Frank Sinatra: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
Beethoven's 2nd: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
The Music of Chance: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
World and Time Enough: 'NoneType' object is not subscriptable
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
Transcriptions: 'imdb_link'
Go Now: 'imdb_link'
Man of the Year: 'NoneType' object is not subscriptable
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_link'
N/A: 'imdb_

In [9]:
print(wiki_file["Running time"].to_string())

0      0 days 01:42:00
1      0 days 01:54:00
2      0 days 01:53:00
3      0 days 01:46:00
4      0 days 01:35:00
5      0 days 01:35:00
6      0 days 01:40:00
7      0 days 01:39:00
8      0 days 00:50:00
9      0 days 01:42:00
10     0 days 01:33:00
11     0 days 01:50:00
12     0 days 02:06:00
13     0 days 02:01:00
14     0 days 01:58:00
15     0 days 01:39:00
16     0 days 01:30:00
17     0 days 01:34:00
18     0 days 01:50:00
19     0 days 03:10:00
20     0 days 01:25:00
21     0 days 01:42:00
22     0 days 02:06:00
23     0 days 01:36:00
24     0 days 01:37:00
25     0 days 01:37:00
26     0 days 01:33:00
27     0 days 01:37:00
28     0 days 00:32:00
29     0 days 01:38:00
30                 NaT
31     0 days 01:35:00
32     0 days 01:38:00
33     0 days 01:24:00
34     0 days 01:41:00
35     0 days 01:37:00
36     0 days 01:26:00
37     0 days 01:39:00
38     0 days 01:37:00
39     0 days 02:18:00
40     0 days 01:39:00
41     0 days 01:25:00
42     0 days 01:31:00
43     0 da

In [17]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
#wiki_movies_df.head()

7042
7042


/var/folders/9s/jwyj784906ndy4td5h3gqtnw0000gn/T/ipykernel_3037/2894390834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
/Users/johncurran/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [11]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [12]:
box_office = wiki_movies_df['Box office'].dropna() 


In [13]:
def is_not_a_string(x): 
    box_office[box_office.map(is_not_a_string)] 
    return type(x) != str

In [14]:
lambda x: type(x) != str 
box_office[box_office.map(lambda x: type(x) != str)]

0        21400000.0
1         2700000.0
10      195300000.0
11       53200000.0
12       15700000.0
           ...     
7036     19400000.0
7037     41900000.0
7038     76100000.0
7039     38400000.0
7040      5500000.0
Name: Box office, Length: 3902, dtype: float64

In [15]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,...,Budget,Box office,Director,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,1990-07-11,0 days 01:42:00,United States,...,$20 million,21400000.0,Renny Harlin,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,1990-05-17,0 days 01:54:00,United States,...,$6 million,2700000.0,James Foley,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,1990-08-10,0 days 01:53:00,United States,...,$35 million,NaN,Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,1990-12-25,0 days 01:46:00,United States,...,$12 million,NaN,Woody Allen,Orion Pictures,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,1990-12-19,0 days 01:35:00,US,...,$25 million,NaN,John Cornell,Paramount Pictures,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018


In [16]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id']